In [1]:
#Import

#pandas
import pandas as pd
from pandas import Series,DataFrame

#numpy
import numpy as np

In [2]:
#路径

path='C:\Users\Wvv\Datacastle_subsidy'

#导入dorm_train数据

dorm_train_name=['id','dorm_date','out_or_in']
dorm_train=pd.read_table(path+'/train/dorm_train.txt',sep=',',header=None,names=dorm_train_name)

#删除重复行并浏览前几行数据
dorm_train=dorm_train.drop_duplicates()
dorm_train.head()

,id,dorm_date,out_or_in
0,13126,2014/01/21 03:31:11,1
1,13126,2014/01/21 04:53:55,0
2,18484,2014/01/21 05:16:18,1
3,296,2014/01/21 05:55:05,1
4,9760,2014/01/21 08:44:53,1


In [3]:
dorm_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2114466 entries, 0 to 2115063
Data columns (total 3 columns):
id           int64
dorm_date    object
out_or_in    int64
dtypes: int64(2), object(1)
memory usage: 64.5+ MB


In [4]:
#定义time转化函数

from datetime import datetime
def time_transform(x):
    return datetime.strptime(x,'%Y/%m/%d %H:%M:%S')

In [5]:
#转换字符串型的time成时间戳格式

dorm_train['dorm_date']=dorm_train['dorm_date'].apply(time_transform)

In [6]:
#时间格式转换

dorm_train['date']=dorm_train['dorm_date'].apply(lambda x:x.strftime('%m-%d-%Y'))
dorm_train['year']=dorm_train['dorm_date'].apply(lambda x:x.strftime('%Y'))
dorm_train['month']=dorm_train['dorm_date'].apply(lambda x:x.strftime('%m'))
dorm_train['hour']=dorm_train['dorm_date'].apply(lambda x:x.strftime('%H'))
dorm_train['weekday']=dorm_train['dorm_date'].apply(lambda x:x.strftime('%A'))

In [7]:
dorm_train.head()

,id,dorm_date,out_or_in,date,year,month,hour,weekday
0,13126,2014-01-21 03:31:11,1,01-21-2014,2014,01,03,Tuesday
1,13126,2014-01-21 04:53:55,0,01-21-2014,2014,01,04,Tuesday
2,18484,2014-01-21 05:16:18,1,01-21-2014,2014,01,05,Tuesday
3,296,2014-01-21 05:55:05,1,01-21-2014,2014,01,05,Tuesday
4,9760,2014-01-21 08:44:53,1,01-21-2014,2014,01,08,Tuesday


In [8]:
#上面数据处理花时很多，先保存处理后的结果

dorm_train.to_csv('dorm_input/dorm_train_time',index=False)

In [9]:
#对hour进行one_hot编码得到hour新表

hour_dummies=pd.get_dummies(dorm_train['hour'])
hour_dummies.columns=['dorm_hour'+x for x in list(hour_dummies.columns)]

#连接两表

dorm_train=dorm_train.join(hour_dummies)

In [10]:
#hour 

#分为4个时段

a=['01','02','03','04','05']
b=['06','07','08','09','10','11']
c=['12','13','14','15','16','17']
d=['18','19','20']
#e=['21','22','23','00']
def get_hour_transform(hour):
    if hour in a:
        return 'a'
    elif hour in b:
        return 'b'
    elif hour in c:
        return 'c'
    elif hour in d:
        return 'd'
    else:
        return 'e'
    
#运用函数
dorm_train['hour']=dorm_train['hour'].apply(get_hour_transform)

#实现one-hot编码
hour_dummies1=pd.get_dummies(dorm_train['hour'])
hour_dummies1.columns=['dorm_hour_01_05','dorm_hour_morning','dorm_hour_afternoon','dorm_hour_evening','dorm_hour_midnight']

#连接两表
dorm_train=dorm_train.join(hour_dummies1)

In [11]:
#weekday

#分为周末与非周末
a=['Monday','Tuesday','Wednesday','Thursday','Friday']
#b=['Saturday','Sunday']
def week_transform(weekday):
    if weekday in a:
        return 'wd'
    else:
        return 'vd'
    
dorm_train['weekday']=dorm_train['weekday'].apply(week_transform)

#对weekday进行one_hot编码得到weekday新表

weekday_dummies = pd.get_dummies(dorm_train['weekday'])
weekday_dummies.columns=['dorm_resting_day','dorm_working_day']
dorm_train=dorm_train.join(weekday_dummies)

In [12]:
#month

#分为在校与假期
#dorm_month=dorm_train.groupby('month')['month'].count()
a=['01','08','02']
def get_month_transform(month):
    if month in a:
        return 'v'
    else:
        return 'o'
    
dorm_train['month']=dorm_train['month'].apply(get_month_transform)

#对month进行one-hot编码
month_dummies=pd.get_dummies(dorm_train['month'])
month_dummies.columns=['dorm_on_school','dorm_vacation']
dorm_train=dorm_train.join(month_dummies)

In [13]:
dorm_train.to_csv('dorm_input/dorm_train.csv',index=False)

In [14]:
dorm_train.head()

,id,dorm_date,out_or_in,date,year,month,hour,weekday,dorm_hour00,dorm_hour01,...,dorm_hour23,dorm_hour_01_05,dorm_hour_morning,dorm_hour_afternoon,dorm_hour_evening,dorm_hour_midnight,dorm_resting_day,dorm_working_day,dorm_on_school,dorm_vacation
0,13126,2014-01-21 03:31:11,1,01-21-2014,2014,v,a,wd,0,0,...,0,1,0,0,0,0,0,1,0,1
1,13126,2014-01-21 04:53:55,0,01-21-2014,2014,v,a,wd,0,0,...,0,1,0,0,0,0,0,1,0,1
2,18484,2014-01-21 05:16:18,1,01-21-2014,2014,v,a,wd,0,0,...,0,1,0,0,0,0,0,1,0,1
3,296,2014-01-21 05:55:05,1,01-21-2014,2014,v,a,wd,0,0,...,0,1,0,0,0,0,0,1,0,1
4,9760,2014-01-21 08:44:53,1,01-21-2014,2014,v,b,wd,0,0,...,0,0,1,0,0,0,0,1,0,1


In [15]:
hour_dummies.head()

,dorm_hour00,dorm_hour01,dorm_hour02,dorm_hour03,dorm_hour04,dorm_hour05,dorm_hour06,dorm_hour07,dorm_hour08,dorm_hour09,...,dorm_hour14,dorm_hour15,dorm_hour16,dorm_hour17,dorm_hour18,dorm_hour19,dorm_hour20,dorm_hour21,dorm_hour22,dorm_hour23
0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
